# Sequential Influence Spread

Influence Spread Using Independent Cascade Model (Weighted Cascade Model)

- Build 'Policy Tree' Reference
- Run Actual Influence Spread that checks reference to attain highest long term potential reward
- Experiment to evaluate performance

## Part 1: Build 'Policy Tree' Reference

## Import Relevant Libraries

In [ ]:
import networkx as nx
import ModelConfig as mc
# import IndependentCascadesModelP001 as icm
# import IndependentCascadesModelP010 as icm
import WeightedCascadeModel as icm
import operator
import random 
import matplotlib.pyplot as plt
import numpy as np
import math

## Greedy Simulation and Selection

In [ ]:
def GreedySelect(g, activated_nodes, model, config, greedy_i):
    ''' 
    Select node with highest average marginal gain
    Variable 'mg' refers to 'marginal gain'
    '''
    mg_dict = {}
    for candidate in g.nodes():
        if candidate in activated_nodes:
            continue
        mg = 0
        for i in range(greedy_i):
            newly_activated_nodes = GreedySim(model, config, candidate)
            mg += len(newly_activated_nodes)
        avg_mg = mg/greedy_i
        mg_dict[candidate] = avg_mg
    influencer = max(mg_dict.items(), key=operator.itemgetter(1))[0]
    print(f"Selected Influencer: Node {influencer}")
    
    return influencer

def GreedySim(model, config, candidate):
    ''' 
    Greedy Simulation to test the influence spread of a given node candidate
    Returns a list of nodes that were activated by the candidate
    '''
    config.add_model_initial_configuration("Infected", [candidate])
    model.set_initial_status(config)
    active_set_size, newly_activated_nodes = model.iteration_bunch()
    newly_activated_nodes.append(candidate)
    model.mg_reset(newly_activated_nodes)
    
    return newly_activated_nodes

## Main Process

In [ ]:
def InitModel(g):
    '''
    Initializes Diffusion Model (Independent Cascade Model)
    Instantiates its configuration
    '''
    model = icm.IndependentCascadesModel(g)
    config = mc.Configuration()
    return model, config

def InfluenceSpread(model, config, influencer):
    '''
    Runs influence spread for the given target set
    Returns its active set size
    '''
    config.add_model_initial_configuration("Infected", [influencer])
    model.set_initial_status(config)
    active_set_size, newly_activated_nodes = model.iteration_bunch()
    return active_set_size, newly_activated_nodes

def get_combination(g, target_set_size, greedy_i=100):
    '''
    Assuming continuous process (previously activated nodes cannot reattempt)
    As compared to one-time influence spread, all influencer same start point
    Default strategy set as greedy 
    '''
    g = remove_isolated_nodes(g)
    model, config = InitModel(g)
    
    all_activated_nodes = []
    influencers = []
    active_ss_list = []
    for i in range(1, target_set_size+1):
        print(f"Selecting Influencer {i}")
        print("------------------------------------")
        
        influencer = GreedySelect(g, all_activated_nodes, model, config, greedy_i)
        influencers.append(influencer)

        active_set_size, newly_activated_nodes = InfluenceSpread(model, config, influencer)
        print(f"Newly Activated Nodes: {newly_activated_nodes}")
        newly_activated_nodes.append(influencer)
        all_activated_nodes.extend(newly_activated_nodes)
        print(f"All Activated Nodes: {all_activated_nodes}")
        print(f"Active Set Size: {active_set_size}")
        
        active_set_size, all_activated_nodes = model.random_deactivation(all_activated_nodes)
        print(f"Final Activated Nodes: {all_activated_nodes}")
        print(f"Final Active Set Size: {active_set_size}")
        print("")

        model.is_reset()
        active_ss_list.append(active_set_size)

    return influencers, active_ss_list[-1]

In [ ]:
def remove_isolated_nodes(g):
    '''
    Returns given graph with no isolated node
    '''
    isolated_nodes = []
    for pair in g.degree:
        node = pair[0]
        degree = pair[1]
        if degree == 0:
            isolated_nodes.append(node)

    for node in isolated_nodes:
        g.remove_node(node)
        
    return g

In [ ]:
# Prepare Graph
'''
nx.erdos_renyi_graph(number of nodes, probability to form edges) 
'''
g = nx.erdos_renyi_graph(150, 0.1) 
g = remove_isolated_nodes(g)

# 'Policy Tree' Reference Settings
'''
target_set_size: The number of influencers to be selected (constant throughout the process)
greedy_i: The number of times greedy simulation is executed (Greedy Iteration)
num_i: The number of times the function get_combination() is repeated to get as many high reward combinations as possible 
LT_ref: A sorted list containing tuple ([influencers], LT_reward)
sorted_influencers: A list containing influencers from LT_ref
'''
target_set_size = 5
greedy_i = 50
LT_ref = []
num_i = 100
for i in range(1, num_i+1):
    print(f"Combination {i}")
    print("------------------------------------")
        
    combination = get_combination(g, target_set_size, greedy_i)
    LT_ref.append(combination)

LT_ref.sort(key=lambda x:x[1], reverse=True)
sorted_influencers = []
for influencers, LT_reward in LT_ref:
    if influencers in sorted_influencers:
        continue  
    sorted_influencers.append(influencers)

In [ ]:
print(LT_ref)
print("")
print(sorted_influencers)

## Part 2: Actual Influence Spread

In [ ]:
def check_reference(sorted_influencers, all_activated_nodes, i, model, config):
    '''
    Check 'Policy Tree' reference to determine next influencer
    Assign influencer to the node in the top combination (highest LT reward)
    If it is already activated, selected another node in the next best matching combination
    Otherwise, if no matching combination, run greedy 
    - wish to avoid this as much as possible especially for large graphs so best to prepare adequate combinations
    Lastly, filter sorted influencers for unmatching combinations
    '''
    influencer = sorted_influencers[0][i]
    j = 1
    while influencer in all_activated_nodes:
        try:   
            influencer = sorted_influencers[j][i]
            j += 1
        except IndexError:
            influencer = GreedySelect(g, all_activated_nodes, model, config, greedy_i)

    print(f"Influencer: Node {influencer}")
    n = []
    for combination in sorted_influencers:
        if combination[i] == influencer:
            n.append(combination)
    print(f"New Sorted Combination List: {n}")
    sorted_influencers = n
    return influencer, sorted_influencers

def greedy_sequential(g, target_set_size, greedy_i, sorted_influencers):
    '''
    Initialize Model
    Select Influencer Greedily 
    - LT Reward Reference, or ST Reward Marginal Gain
    Run Influence Spread
    Execute Random Deactivation
    '''
    g = remove_isolated_nodes(g)
    model, config = InitModel(g)
    
    all_activated_nodes = []
    influencers = []
    active_ss_list = []
    
    for i in range(target_set_size):
        print(f"Selecting Influencer {i+1}")
        print("------------------------------------")
        
        if sorted_influencers == []: # If no matching combination, simulate greedy
            influencer = GreedySelect(g, all_activated_nodes, model, config, greedy_i)
            print("Approach: Greedy Select")
        else: # Check LT Reward Reference
            influencer, sorted_influencers = check_reference(sorted_influencers, all_activated_nodes, i, model, config)
            print("Approach: LT Reward Reference")
            
        influencers.append(influencer)

        active_set_size, newly_activated_nodes = InfluenceSpread(model, config, influencer)
        print(f"Newly Activated Nodes: {newly_activated_nodes}")
        newly_activated_nodes.append(influencer)
        all_activated_nodes.extend(newly_activated_nodes)
        print(f"All Activated Nodes: {all_activated_nodes}")
        print(f"Active Set Size: {active_set_size}")
        
        active_set_size, all_activated_nodes = model.random_deactivation(all_activated_nodes)
        print(f"Final Activated Nodes: {all_activated_nodes}")
        print(f"Final Active Set Size: {active_set_size}")
        print("")

        model.is_reset()
        active_ss_list.append(active_set_size)

    return influencers, active_ss_list[-1]

In [ ]:
# Experiment Settings for Adaptive Greedy 
'''
exp_i: Experiment Iterations, the number of times the experiment is repeated in order to get the average activated nodes
'''
exp_i = 20
greedy_i = 50
target_set_size = 5
g_total_activated_nodes = 0

for i in range(exp_i):
    influencers, active_set_size = greedy_sequential(g, target_set_size, greedy_i, sorted_influencers)
    g_total_activated_nodes += active_set_size

g_avg_activated_nodes = g_total_activated_nodes / exp_i

## Part 3: Experiment by comparing to baseline (Random)

In [ ]:
def random_sequential(g, target_set_size):
    '''
    Initialize Model
    Select Influencer Randomly 
    Run Influence Spread
    Execute Random Deactivation
    '''
    g = remove_isolated_nodes(g)
    model, config = InitModel(g)
    
    all_activated_nodes = []
    influencers = []
    active_ss_list = []
    
    for i in range(target_set_size):
        print(f"Selecting Influencer {i+1}")
        print("------------------------------------")
        
        influencer = random.randint(0, len(g)-1)
        while influencer in all_activated_nodes:
            influencer = random.randint(0, len(g)-1)
        print(f"Influencer: Node {influencer}")
            
        influencers.append(influencer)

        active_set_size, newly_activated_nodes = InfluenceSpread(model, config, influencer)
        print(f"Newly Activated Nodes: {newly_activated_nodes}")
        newly_activated_nodes.append(influencer)
        all_activated_nodes.extend(newly_activated_nodes)
        print(f"All Activated Nodes: {all_activated_nodes}")
        print(f"Active Set Size: {active_set_size}")
        
        active_set_size, all_activated_nodes = model.random_deactivation(all_activated_nodes)
        print(f"Final Activated Nodes: {all_activated_nodes}")
        print(f"Final Active Set Size: {active_set_size}")
        print("")

        model.is_reset()
        active_ss_list.append(active_set_size)

    return influencers, active_ss_list[-1]

In [ ]:
# Experiment Settings for Random
'''
exp_i: Experiment Iterations, the number of times the experiment is repeated in order to get the average activated nodes
'''
exp_i = 20
target_set_size = 5
r_total_activated_nodes = 0

for i in range(exp_i):
    influencers, active_set_size = random_sequential(g, target_set_size)
    r_total_activated_nodes += active_set_size

r_avg_activated_nodes = r_total_activated_nodes / exp_i

In [ ]:
print(g_avg_activated_nodes)
print(r_avg_activated_nodes)